In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from google.colab import drive
from sklearn.preprocessing import StandardScaler
from google.colab import drive
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import xgboost as xgb
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# csv locations can be updated
df_html_0 = pd.read_csv("/content/drive/MyDrive/dataminingPhising/features/html_features_0_10000.csv")  # Load your dataset
df_html_10000 = pd.read_csv("/content/drive/MyDrive/dataminingPhising/features/html_features_0_10000.csv")  # Load your dataset
df_html_20000 = pd.read_csv("/content/drive/MyDrive/dataminingPhising/features/html_features_10000_20000.csv")  # Load your dataset
df_html_30000 = pd.read_csv("/content/drive/MyDrive/dataminingPhising/features/html_features_20000_30000.csv")  # Load your dataset

In [ ]:
df_features_0 = pd.read_csv("/content/drive/MyDrive/dataminingPhising/features/url_features_0_10000.csv")
df_features_10000 = pd.read_csv("/content/drive/MyDrive/dataminingPhising/features/url_features_10000_20000.csv")
df_features_20000 = pd.read_csv("/content/drive/MyDrive/dataminingPhising/features/url_features_20000_30000.csv")
df_features_30000 = pd.read_csv("/content/drive/MyDrive/dataminingPhising/features/url_features_30000_80000.csv")

In [ ]:
# Merge all dataframes into one
merged_df_html = pd.concat([df_html_0 , df_html_10000, df_html_20000, df_html_30000], ignore_index=True)
merged_df_features = pd.concat([df_features_0, df_features_10000, df_features_20000, df_features_30000], ignore_index=True)


In [ ]:
merged_df_html.drop('rec_id', axis=1, inplace=True)
merged_df_features.drop('rec_id', axis=1, inplace=True)

In [ ]:
def preprocess(train_df):

  # Split labels and features
  labels = train_df['phishing']
  features = train_df.drop('phishing', axis=1)

  # Split the data into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.4, random_state=42)

  # Standardize the features
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)


  return X_train, y_train, X_test, y_test



In [ ]:
html_train_X, html_train_y, html_test_X, html_test_y = preprocess(merged_df_html)
features_train_X, features_train_y, features_test_X, features_test_y = preprocess(merged_df_features)
#

In [ ]:
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "k-NN (k=1)": KNeighborsClassifier(n_neighbors=1),
    "k-NN (k=3)": KNeighborsClassifier(n_neighbors=3),
    "Gaussian Naive Bayes": GaussianNB(),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "MLP Neural Network": MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": xgb.XGBClassifier(random_state=42)
}

In [ ]:
def evaluate_models(models, X_train, y_train, X_test, y_test):
  for model_name, model in models.items():
      # Train the model
      model.fit(features_train_X, features_train_y)

      # Predict on the training set
      y_train_pred = model.predict(features_train_X)
      # Calculate training accuracy
      train_accuracy = accuracy_score(features_train_y, y_train_pred)

      # Predict on the test set
      y_test_pred = model.predict(features_test_X)
      # Calculate testing accuracy
      test_accuracy = accuracy_score(features_test_y, y_test_pred)
      print(f"ID of {model_name}: {id(model)}")  # Print the ID

      # Print the accuracies for each model
      print(f"{model_name} - Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")

In [ ]:
evaluate_models(models, features_train_X, features_train_y, features_test_X, features_test_y)

ID of Decision Tree: 135334041494928
Decision Tree - Train Accuracy: 0.9836, Test Accuracy: 0.8803
ID of k-NN (k=1): 135334041497040
k-NN (k=1) - Train Accuracy: 0.9781, Test Accuracy: 0.8906
ID of k-NN (k=3): 135334041497088
k-NN (k=3) - Train Accuracy: 0.9364, Test Accuracy: 0.8948
ID of Gaussian Naive Bayes: 135334041497136
Gaussian Naive Bayes - Train Accuracy: 0.7131, Test Accuracy: 0.7123
ID of Logistic Regression: 135334041493968
Logistic Regression - Train Accuracy: 0.8456, Test Accuracy: 0.8466
ID of MLP Neural Network: 135334041486864
MLP Neural Network - Train Accuracy: 0.9518, Test Accuracy: 0.9080
ID of Random Forest: 135334041489504
Random Forest - Train Accuracy: 0.9836, Test Accuracy: 0.9158
ID of XGBoost: 135334041486720
XGBoost - Train Accuracy: 0.9323, Test Accuracy: 0.9150


array([[-0.20851214, -0.6244988 , -0.22609012, ..., -0.10972259,
        -0.43253947,  0.20000458],
       [-0.89995137, -0.25904994,  3.29139662, ..., -0.10972259,
        -0.43253947, -1.19032417],
       [-0.20851214,  0.10639891, -0.22609012, ..., -0.10972259,
        -0.43253947,  0.20000458],
       ...,
       [-0.20851214, -0.6244988 , -0.22609012, ..., -0.10972259,
        -0.43253947,  0.20000458],
       [ 1.17436632, -0.6244988 , -0.22609012, ..., -0.10972259,
        -0.43253947,  2.9806621 ],
       [-0.20851214, -0.6244988 , -0.22609012, ..., -0.10972259,
        -0.43253947,  0.20000458]])

In [ ]:
models2 = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "k-NN (k=1)": KNeighborsClassifier(n_neighbors=1),
    "k-NN (k=3)": KNeighborsClassifier(n_neighbors=3),
    "Gaussian Naive Bayes": GaussianNB(),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "MLP Neural Network": MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": xgb.XGBClassifier(random_state=42)
}

In [ ]:
columns_to_remove = [
    'qty_/_domain', 'qty_?_domain', 'qty_=_domain', 'qty_@_domain',
    'qty_&_domain', 'qty_!_domain', 'qty_ _domain', 'qty_~_domain',
    'qty_,_domain', 'qty_+_domain', 'qty_*_domain', 'qty_#_domain',
    'qty_$_domain', 'qty_%_domain'  # ... add any other columns with only '0' values
]


merged_df_features.drop(columns=columns_to_remove, inplace=True)

features_train_X, features_train_y, features_test_X, features_test_y = preprocess(merged_df_features)

evaluate_models(models2, features_train_X, features_train_y, features_test_X, features_test_y)


ID of Decision Tree: 135333808204016
Decision Tree - Train Accuracy: 0.9836, Test Accuracy: 0.8823
ID of k-NN (k=1): 135333808199168
k-NN (k=1) - Train Accuracy: 0.9781, Test Accuracy: 0.8904
ID of k-NN (k=3): 135333808199408
k-NN (k=3) - Train Accuracy: 0.9364, Test Accuracy: 0.8946
ID of Gaussian Naive Bayes: 135333808200896
Gaussian Naive Bayes - Train Accuracy: 0.7131, Test Accuracy: 0.7123
ID of Logistic Regression: 135333808196384
Logistic Regression - Train Accuracy: 0.8456, Test Accuracy: 0.8466
ID of MLP Neural Network: 135333808197008
MLP Neural Network - Train Accuracy: 0.9475, Test Accuracy: 0.9102
ID of Random Forest: 135333808197392
Random Forest - Train Accuracy: 0.9835, Test Accuracy: 0.9152
ID of XGBoost: 135333808198784
XGBoost - Train Accuracy: 0.9323, Test Accuracy: 0.9150


In [ ]:
# decision_tree = DecisionTreeClassifier(criterion='entropy', min_samples_split=4, random_state=32)
# decision_tree.fit(features_train_X, features_train_y)

# # Calculate accuracy on the test set
# y_pred = decision_tree.predict(features_test_X)
# accuracy = accuracy_score(features_test_y, y_pred)
# print(f"Accuracy: {accuracy}")

In [ ]:
# decision_tree_2 = DecisionTreeClassifier(criterion='entropy', min_samples_split=32, random_state=32)
# decision_tree_2.fit(features_train_X, features_train_y)

# # Calculate accuracy on the test set
# y_pred = decision_tree_2.predict(features_test_X)
# accuracy = accuracy_score(features_test_y, y_pred)
# print(f"Accuracy: {accuracy}")

In [ ]:
# def preprocess_2(train_df):

#   # Split labels and features
#   labels = train_df['phishing']
#   features = train_df.drop('phishing', axis=1)

#   # Split the data into training and testing sets
#   X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.4, random_state=42)


#   # Standardize the features
#   scaler = StandardScaler()
#   X_train = scaler.fit_transform(X_train)
#   X_test = scaler.transform(X_test)


#   return X_train, y_train, X_test, y_test